In [2]:
import os
import typing

import numpy as np
import pandas as pd
from scipy.stats import rankdata
from itertools import product


pd.set_option("chained_assignment", None)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", 5000)
pd.set_option("display.width", 10000)

In [12]:
algos=["algo_1","alog_2"]
seeds=list(range(50))
benchmarks=["bench_1","bench_2"]

combinations = product(algos, benchmarks, [0],seeds)
toy_df = pd.DataFrame(combinations, columns=["algorithm", "benchmark", "value", "seed"])

def gen_value_rand(row):
    return np.random.rand()
toy_df["value"] = toy_df.apply(gen_value_rand, axis=1)
print(toy_df)

    algorithm benchmark     value  seed
0      algo_1   bench_1  0.352754     0
1      algo_1   bench_1  0.340719     1
2      algo_1   bench_1  0.471027     2
3      algo_1   bench_1  0.395386     3
4      algo_1   bench_1  0.473410     4
5      algo_1   bench_1  0.632317     5
6      algo_1   bench_1  0.465834     6
7      algo_1   bench_1  0.470288     7
8      algo_1   bench_1  0.037996     8
9      algo_1   bench_1  0.142902     9
10     algo_1   bench_1  0.236929    10
11     algo_1   bench_1  0.043768    11
12     algo_1   bench_1  0.867997    12
13     algo_1   bench_1  0.902343    13
14     algo_1   bench_1  0.232504    14
15     algo_1   bench_1  0.392970    15
16     algo_1   bench_1  0.080859    16
17     algo_1   bench_1  0.269909    17
18     algo_1   bench_1  0.586705    18
19     algo_1   bench_1  0.407124    19
20     algo_1   bench_1  0.143022    20
21     algo_1   bench_1  0.264365    21
22     algo_1   bench_1  0.716828    22
23     algo_1   bench_1  0.758149    23
